In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def scrapy_player_data(name):
    name = name.replace(" ", "+")
    url = f"https://www.tennisexplorer.com/list-players/?search-text-pl={name}&country="
    page = requests.get(url)
   
    find = False
    for i in page.text.split(">"):
        if "/player" in i:
            if not find:
                to_search = i.split("/")[2]
                find = True
    url2 = f"https://www.tennisexplorer.com/player/{to_search}/"
    page = requests.get(url2)
    soup = BeautifulSoup(page.text, "html.parser")
    scrapy_data = soup.find_all("div",{"class": "date"})
    d = dict(str(s).replace("<div class=\"date\">", "").replace("</div>", "").split(': ') for s in scrapy_data)
    res_dt = {}
    res_dt["Country"] = d["Country"]
    res_dt["Height"] = d["Height / Weight"].replace(" cm / ", ",").replace(" kg", "").split(",")[0]
    tmp = d["Age"].replace(". ", "/").replace(")", "").split(" (")
    res_dt["Age"] = tmp[0]
    res_dt["Sex"] = d["Sex"]
    res_dt["Plays"] = d["Plays"]
    res_dt["YearOfBirth"] = tmp[1]
    return res_dt

In [3]:
import time

In [4]:
start_time = time.time()
player_data = scrapy_player_data("Daniil Medved")
print("--- Python scraping time = %s seconds ---" % (time.time() - start_time))
print(player_data)

--- Python scraping time = 0.5068256855010986 seconds ---
{'Country': 'Russia', 'Height': '198', 'Age': '26', 'Sex': 'man', 'Plays': 'right', 'YearOfBirth': '11/2/1996'}


In [5]:
from julia import Main
Main.include("player_scraper.jl")

<PyCall.jlwrap scrapy>

In [15]:
start_time = time.time()
player_data = Main.scrapy("Daniil Medvedev")
print("--- Julia scraping time = %s seconds ---" % (time.time() - start_time))
print(player_data)

--- Julia scraping time = 0.662243127822876 seconds ---
<PyCall.jlwrap {"Country" = " Russia", "Height" = "198", "Age" = "26", "Sex" = " man", "Plays" = " right", "YearOfBirth" = "11/2/1996"}>


In [11]:
import rpy2.robjects as robjects

In [12]:
start_time = time.time()
robjects.r['source']('player_scraper.R')
scraped_data_r = robjects.r['scrapy']("Daniil Medvedev")
print("--- R scraping time = %s seconds ---" % (time.time() - start_time))

--- R scraping time = 0.5229387283325195 seconds ---


In [13]:
def convert_rlist_to_dict(scraped_data_r):
    scraped_data = []
    for i in scraped_data_r:
        scraped_data.append(i[0])
    data_dict = {"Country": scraped_data[0], "Height": scraped_data[1],"Age": scraped_data[2],
                 "YearOfBirth": scraped_data[3]+"/"+scraped_data[4]+"/"+scraped_data[5],
                "Sex": scraped_data[6], "Plays": scraped_data[7]}
    return data_dict

In [14]:
data_dict = convert_rlist_to_dict(scraped_data_r)
data_dict

{'Country': 'Russia',
 'Height': 198.0,
 'Age': '26',
 'YearOfBirth': '11/2/1996',
 'Sex': 'man',
 'Plays': 'right'}